In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from spmf import Spmf
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import ast
pd.options.plotting.backend = "plotly"
import plotly.express as px
# from chart_studio import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.subplots as sp
pd.options.display.max_colwidth = None
pd.set_option('display.max_rows', 70)
from pandas.plotting import table 
import nltk

# N-grams
In this notebook the frequent patterns will be investigated using a text analytical approach, centered around N-grams. This provide an alternative to using the frequent pattern mining algorithm VMSP.

## Outline
1. Full data set\
    1.1 0.75 %
2. Data set with 70% coverage.\
    2.1 with merged repeating apps, after removal.
3. Individual perspectiv\
    3.1 frequent apps

## Full data set
In the following we will start out by working with the full data set. The data set has already been divided into sessions based on a 293 seconds sleep time threshold, and any repeating apps have been merged. There are 76775122 sessions which in the following analysis each is represented as a document, containt a string of the apps package names which are seperated by whitespace.

In [2]:
data=pd.read_csv("sorted_coded_session_293s_merged.csv")

In [3]:
user_corpus=data.groupby(['user_id','session']).package_name.apply(' '.join)

### Minimum frequency comparative to VMSP
In the same way as the frequent pattern mining algorithm has the minimum support parameter, a minimum frequency will be used to filter the obtained patterns. In the following section the minimum frequency corresponds to 136450 occurences. This is the same number of seesions needed to obtain minimum support of 0.75 when running the VMSP on "293sec_sessions_70_coverage_without_length_1.txt"

#### 1-gram and frequent apps
The first step is to take a look at the 1-grams, which corresponds to finding the frequent apps.

In [4]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1))

In [5]:
vectorization=vectorizer.fit_transform(user_corpus.values)

In [149]:
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>136450)[1]]

array(['bbc.mobile.news.uk', 'bbc.mobile.weather',
       'co.uk.nationwide.mobile', 'com.accuweather.android',
       'com.amazon.kindle', 'com.amazon.mp3',
       'com.amazon.mshop.android.shopping', 'com.android.chrome',
       'com.android.email', 'com.android.incallui', 'com.android.phone',
       'com.audible.application',
       'com.barclays.android.barclaysmobilebanking', 'com.bskyb.fbscore',
       'com.bskyb.skynews.android', 'com.bskyb.sportnews',
       'com.dailymail.online', 'com.discord', 'com.ebay.mobile',
       'com.facebook.katana', 'com.facebook.orca',
       'com.fitbit.fitbitmobile', 'com.google.android.apps.docs',
       'com.google.android.apps.inbox', 'com.google.android.apps.maps',
       'com.google.android.apps.messaging',
       'com.google.android.apps.photos', 'com.google.android.calendar',
       'com.google.android.gm', 'com.google.android.gms',
       'com.google.android.googlequicksearchbox',
       'com.google.android.keep', 'com.google.android.musi

In [152]:
print("Number of frequent apps:",vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>136450)[1]].shape[0])

Number of frequent apps: 87


Since we are here able to work with the full data set a 0.75 support percentage is not neccessarily the same.

### Minimum frequency of 0.75% of full data set

In [63]:
print("number of sessions:",user_corpus.shape[0])
print("0.75% of sessions:",user_corpus.shape[0]/100*0.75)

number of sessions: 76775122
0.75% of sessions: 575813.415


#### 1-grams and frequent apps
Since the 1-grams where already determined in the previous section, only the filtering needs to be updated to match the overall 0.75% minimum. Based on the above calculations, the minimum is determined to correspond to 575814 sessions.

In [6]:
frequent_apps=vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>575814)[1]]
frequent_apps

array(['bbc.mobile.news.uk', 'com.amazon.mshop.android.shopping',
       'com.android.chrome', 'com.android.incallui', 'com.android.phone',
       'com.ebay.mobile', 'com.facebook.katana', 'com.facebook.orca',
       'com.google.android.apps.maps',
       'com.google.android.apps.messaging', 'com.google.android.calendar',
       'com.google.android.gm', 'com.google.android.googlequicksearchbox',
       'com.google.android.youtube', 'com.instagram.android',
       'com.microsoft.office.outlook', 'com.nianticlabs.pokemongo',
       'com.snapchat.android', 'com.sony.nfx.app.sfrc',
       'com.sonyericsson.album',
       'com.sonyericsson.android.socialphonebook',
       'com.sonyericsson.conversations', 'com.sonyericsson.music',
       'com.sonymobile.calendar', 'com.sonymobile.email',
       'com.spotify.music', 'com.twitter.android', 'com.whatsapp',
       'com.yahoo.mobile.client.android.mail'], dtype=object)

In [9]:
print("Number of frequent apps:",len(frequent_apps))

Number of frequent apps: 29


With 29 apps the number of found sessions are much lower, but also more comparable to the 25 frequenct apps found with VMSP. Furthermore almost all frequent apps found with VMSP are included.

#### 2-grams
We now investigate which patterns of length two occrue with at least the minimum frequency.

In [154]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2))

In [155]:
vectorization=vectorizer.fit_transform(user_corpus.values)

In [158]:
print("Number of 2-grams:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>575814)[1]]))

Number of 2-grams: 29


In [160]:
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>575814)[1]]

array(['com.android.chrome com.facebook.katana',
       'com.android.chrome com.google.android.googlequicksearchbox',
       'com.android.chrome com.whatsapp',
       'com.android.incallui com.sonyericsson.android.socialphonebook',
       'com.facebook.katana com.android.chrome',
       'com.facebook.katana com.facebook.orca',
       'com.facebook.katana com.google.android.googlequicksearchbox',
       'com.facebook.katana com.instagram.android',
       'com.facebook.katana com.snapchat.android',
       'com.facebook.katana com.sonyericsson.conversations',
       'com.facebook.katana com.whatsapp',
       'com.facebook.orca com.facebook.katana',
       'com.google.android.gm com.android.chrome',
       'com.google.android.googlequicksearchbox com.android.chrome',
       'com.google.android.googlequicksearchbox com.facebook.katana',
       'com.google.android.googlequicksearchbox com.whatsapp',
       'com.instagram.android com.facebook.katana',
       'com.instagram.android com.whatsap

As for the 1-grams we also here recongnise serveral of the patterns from the VMPS output.

#### 3-grams
At last we look at the patterns of length 3

In [161]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3))
vectorization=vectorizer.fit_transform(user_corpus.values)

In [162]:
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>575814)[1]]

array(['com.google.android.googlequicksearchbox com.android.chrome com.google.android.googlequicksearchbox',
       'com.whatsapp com.facebook.katana com.whatsapp'], dtype=object)

In [163]:
print("Number of 3-grams:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>575814)[1]]))

Number of 3-grams: 2


None of the patterns of length 3 were found with VMSP. Furthermore it is observed that for all 4 patterns, the apps in one patterns are similar in category.

#### 4-grams
Below in can be seen that the data set does not contain any patterns of legnth four, so all frequent patterns has been found.

In [164]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4))
vectorization=vectorizer.fit_transform(user_corpus.values)

In [165]:
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>575814)[1]]

array([], dtype=object)

In [166]:
print("Number of 4-grams:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>575814)[1]]))

Number of 4-grams: 0


## Data set with 70% individual coverage.
For direct comparison the N-grams are found for the same data set as the VMSP was run on.

In [61]:
data_coverage_70=pd.read_csv("data_coverage_70.csv")

In [62]:
print("Number of number of observations:",data_coverage_70.shape[0])
print("Number of different apps left:",data_coverage_70.app_code.nunique())

Number of number of observations: 90303962
Number of different apps left: 2268


In [67]:
data_coverage_70

,package_name,user_id,start_time,end_time,time_spend_x,app_code,sequence_number,sequence_Start_time,sequence_end_time,sleep_time,new_session,session,time_spend_y,time_spend_total,coverage,session_length
0,com.android.chrome,0,441,1012,571,7649,2,441,1012,0,False,0,831069,1688521,0.492188,2
1,com.google.android.youtube,0,1582,1712,130,23967,6,1582,1712,10,False,0,1149788,1688521,0.680944,2
2,com.android.chrome,0,3926,4076,150,7649,12,3926,4076,0,False,2,831069,1688521,0.492188,1
3,com.android.chrome,0,32291,32769,478,7649,24,32291,32769,40,False,6,831069,1688521,0.492188,2
4,com.google.android.youtube,0,32091,32101,10,23967,22,32091,32101,2142,True,6,1149788,1688521,0.680944,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90303957,com.google.android.youtube,9999,26588896,26588966,70,23967,13303,26588896,26588966,754,True,5100,1002729,1360485,0.737038,1
90303958,com.google.android.youtube,9999,26628994,26629112,118,23967,13333,26628994,26629112,339,True,5111,1002729,1360485,0.737038,1
90303959,com.google.android.youtube,9999,26703610,26703880,270,23967,13368,26703610,26703880,3073,True,5125,1002729,1360485,0.737038,1
90303960,com.google.android.youtube,9999,26704637,26704647,10,23967,13369,26704637,26704647,757,True,5126,1002729,1360485,0.737038,1


Since the VMSP was only running on sessions with more than one app, all sessions of length one is filtered out.

In [168]:
data_coverage_70_min_length_2=data_coverage_70[data_coverage_70.session_length>1]

In [169]:
user_corpus_70=data_coverage_70_min_length_2.groupby(['user_id','session']).package_name.apply(' '.join)

In [170]:
print("Number of sessions:",user_corpus_70.shape[0])
print("0.75% of sessions:",user_corpus_70.shape[0]/100*0.75)

Number of sessions: 18193255
0.75% of sessions: 136449.41249999998


### Frequent apps (1-gram)

In [142]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array(['bbc.mobile.news.uk', 'com.android.chrome', 'com.android.incallui',
       'com.ebay.mobile', 'com.facebook.katana', 'com.facebook.orca',
       'com.google.android.apps.maps',
       'com.google.android.apps.messaging', 'com.google.android.gm',
       'com.google.android.googlequicksearchbox',
       'com.google.android.youtube', 'com.instagram.android',
       'com.microsoft.office.outlook', 'com.nianticlabs.pokemongo',
       'com.snapchat.android', 'com.sonyericsson.album',
       'com.sonyericsson.android.socialphonebook',
       'com.sonyericsson.conversations', 'com.sonymobile.email',
       'com.spotify.music', 'com.twitter.android', 'com.whatsapp',
       'com.yahoo.mobile.client.android.mail'], dtype=object)

In [143]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 23


Here the same apps are found to be frequent as with VMSP.

### 2-grams

In [172]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array(['com.android.chrome com.android.chrome',
       'com.android.chrome com.facebook.katana',
       'com.android.chrome com.facebook.orca',
       'com.android.chrome com.google.android.googlequicksearchbox',
       'com.android.chrome com.google.android.youtube',
       'com.android.chrome com.instagram.android',
       'com.android.chrome com.sonyericsson.conversations',
       'com.android.chrome com.whatsapp',
       'com.android.incallui com.android.incallui',
       'com.facebook.katana com.android.chrome',
       'com.facebook.katana com.facebook.katana',
       'com.facebook.katana com.facebook.orca',
       'com.facebook.katana com.instagram.android',
       'com.facebook.katana com.sonyericsson.conversations',
       'com.facebook.katana com.whatsapp',
       'com.facebook.orca com.android.chrome',
       'com.facebook.orca com.facebook.orca',
       'com.google.android.gm com.google.android.gm',
       'com.google.android.googlequicksearchbox com.android.chrome',
       

In [174]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 39


### 3-grams

In [175]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array(['com.android.chrome com.android.chrome com.android.chrome',
       'com.android.chrome com.whatsapp com.whatsapp',
       'com.facebook.katana com.facebook.katana com.android.chrome',
       'com.facebook.katana com.facebook.katana com.facebook.katana',
       'com.facebook.orca com.facebook.orca com.facebook.orca',
       'com.google.android.googlequicksearchbox com.android.chrome com.android.chrome',
       'com.google.android.googlequicksearchbox com.google.android.googlequicksearchbox com.android.chrome',
       'com.google.android.googlequicksearchbox com.google.android.googlequicksearchbox com.google.android.googlequicksearchbox',
       'com.instagram.android com.instagram.android com.instagram.android',
       'com.snapchat.android com.snapchat.android com.snapchat.android',
       'com.sonyericsson.conversations com.sonyericsson.conversations com.sonyericsson.conversations',
       'com.whatsapp com.whatsapp com.android.chrome',
       'com.whatsapp com.whatsapp com.wha

In [176]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 13


### 4-grams

In [179]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(4,4))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array(['com.android.chrome com.android.chrome com.android.chrome com.android.chrome',
       'com.facebook.katana com.facebook.katana com.facebook.katana com.facebook.katana',
       'com.facebook.orca com.facebook.orca com.facebook.orca com.facebook.orca',
       'com.google.android.googlequicksearchbox com.google.android.googlequicksearchbox com.android.chrome com.android.chrome',
       'com.google.android.googlequicksearchbox com.google.android.googlequicksearchbox com.google.android.googlequicksearchbox com.google.android.googlequicksearchbox',
       'com.sonyericsson.conversations com.sonyericsson.conversations com.sonyericsson.conversations com.sonyericsson.conversations',
       'com.whatsapp com.whatsapp com.whatsapp com.whatsapp'],
      dtype=object)

In [180]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 7


### 5.grams

In [181]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(5,5))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array(['com.whatsapp com.whatsapp com.whatsapp com.whatsapp com.whatsapp'],
      dtype=object)

In [182]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 1


### 6-grams

In [183]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(6,6))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array([], dtype=object)

In [184]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 0


### Mereged repeating apps

In [187]:
data_coverage_70["previous_app_code"]=data_coverage_70.groupby(["user_id","session"]).app_code.shift(1)
data_merge=data_coverage_70.copy()
data_merge=data_merge[(data_merge.previous_app_code==data_merge.app_code) | (data_merge.app_code==data_merge.groupby(['user_id','session']).app_code.shift(-1))]
data_merge['new_event'] = data_merge.previous_app_code!=data_merge.app_code
data_merge['event'] = data_merge.groupby(['user_id','session']).new_event.cumsum().astype('int')

In [189]:
data2_grouped = data_merge.groupby(['user_id', 'session','event']).agg({'end_time':'max'})

data2_grouped = data2_grouped.reset_index()

data2_grouped = data2_grouped.rename(columns={'end_time':'event_end_time'})

data3 = pd.merge(data_merge, data2_grouped, how='left', on=['user_id', 'session','event'])

data3 = data3[data3['end_time'] == data3['event_end_time']]

#because in the same event more apps can be closed at once.
data3=data3.drop_duplicates(subset=['user_id', 'session','event'], keep='last')
max_index=data_merge.iloc[data3.index].index

In [190]:
data2_grouped = data_merge.groupby(['user_id', 'session','event']).agg({'start_time':'min'})

data2_grouped = data2_grouped.reset_index()

data2_grouped = data2_grouped.rename(columns={'start_time':'event_start_time'})

data3 = pd.merge(data_merge, data2_grouped, how='left', on=['user_id', 'session','event'])

data3 = data3[data3['start_time'] == data3['event_start_time']]

#because in the same event more apps can be opened at once.
data3=data3.drop_duplicates(subset=['user_id', 'session','event'], keep='last')
min_index=data_merge.iloc[data3.index].index

In [192]:
data3=data_coverage_70.copy()
data3.loc[max_index,"start_time"]=data3.loc[min_index].set_index(max_index).start_time
data3=data3.drop(set(data_merge.index)-set(max_index))
len(data_coverage_70.index)-(len(data_merge.index)-len(max_index))==len(data3.index)

True

In [193]:
data3['time_spend']=data3['end_time']-data3['start_time']
data3=data3.drop('previous_app_code',axis=1)
data3['sleep_time']=data3.start_time-data3.sequence_end_time.shift(1,fill_value=0)
data3.loc[data3[data3['sleep_time']<0].index,'sleep_time']=0
data3=data3.reset_index(drop=True)

In [194]:
data3.to_csv("70_coverage_merged.csv",index=False)

In [195]:
data3=data3[data3.session_length>1]

In [196]:
user_corpus_70=data3.groupby(['user_id','session']).package_name.apply(' '.join)

In [197]:
print("Number of sessions:",user_corpus_70.shape[0])
print("0.75% of sessions:",user_corpus_70.shape[0]/100*0.75)

Number of sessions: 18193255
0.75% of sessions: 136449.41249999998


#### 1-grams

In [199]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(1,1))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array(['bbc.mobile.news.uk', 'com.android.chrome', 'com.android.incallui',
       'com.ebay.mobile', 'com.facebook.katana', 'com.facebook.orca',
       'com.google.android.apps.maps',
       'com.google.android.apps.messaging', 'com.google.android.gm',
       'com.google.android.googlequicksearchbox',
       'com.google.android.youtube', 'com.instagram.android',
       'com.microsoft.office.outlook', 'com.nianticlabs.pokemongo',
       'com.snapchat.android', 'com.sonyericsson.album',
       'com.sonyericsson.android.socialphonebook',
       'com.sonyericsson.conversations', 'com.sonymobile.email',
       'com.spotify.music', 'com.twitter.android', 'com.whatsapp',
       'com.yahoo.mobile.client.android.mail'], dtype=object)

In [200]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 23


In [201]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(2,2))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array(['com.android.chrome com.facebook.katana',
       'com.android.chrome com.facebook.orca',
       'com.android.chrome com.google.android.googlequicksearchbox',
       'com.android.chrome com.google.android.youtube',
       'com.android.chrome com.instagram.android',
       'com.android.chrome com.sonyericsson.conversations',
       'com.android.chrome com.whatsapp',
       'com.facebook.katana com.android.chrome',
       'com.facebook.katana com.facebook.orca',
       'com.facebook.katana com.instagram.android',
       'com.facebook.katana com.sonyericsson.conversations',
       'com.facebook.katana com.whatsapp',
       'com.facebook.orca com.android.chrome',
       'com.google.android.googlequicksearchbox com.android.chrome',
       'com.instagram.android com.android.chrome',
       'com.instagram.android com.facebook.katana',
       'com.instagram.android com.snapchat.android',
       'com.instagram.android com.whatsapp',
       'com.snapchat.android com.instagram.android',
   

In [202]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 25


In [203]:
vectorizer = CountVectorizer(analyzer='word',tokenizer=lambda text: re.split(' ',text), ngram_range=(3,3))
vectorization=vectorizer.fit_transform(user_corpus_70.values)
vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]

array([], dtype=object)

In [204]:
print("Number of frequent patterns:",len(vectorizer.get_feature_names_out()[np.where((vectorization>0).sum(axis=0)>=136450)[1]]))

Number of frequent patterns: 0


# Individual perspective
In this section the individual users usage is compared with the overall persepctive.
## Frequently used apps
Initially we take a closer look at the frequent apps found as the 1-grams for the whole data set which occur in at least 0.75%.

In [11]:
frequent_apps

array(['bbc.mobile.news.uk', 'com.amazon.mshop.android.shopping',
       'com.android.chrome', 'com.android.incallui', 'com.android.phone',
       'com.ebay.mobile', 'com.facebook.katana', 'com.facebook.orca',
       'com.google.android.apps.maps',
       'com.google.android.apps.messaging', 'com.google.android.calendar',
       'com.google.android.gm', 'com.google.android.googlequicksearchbox',
       'com.google.android.youtube', 'com.instagram.android',
       'com.microsoft.office.outlook', 'com.nianticlabs.pokemongo',
       'com.snapchat.android', 'com.sony.nfx.app.sfrc',
       'com.sonyericsson.album',
       'com.sonyericsson.android.socialphonebook',
       'com.sonyericsson.conversations', 'com.sonyericsson.music',
       'com.sonymobile.calendar', 'com.sonymobile.email',
       'com.spotify.music', 'com.twitter.android', 'com.whatsapp',
       'com.yahoo.mobile.client.android.mail'], dtype=object)

First the overall "distribution is calculated", corresponding to the number of sessions each app occur in.

In [87]:
fig = px.bar(x=frequent_apps, y = [(vectorization>0).sum(axis=0)[0,np.where(vectorizer.get_feature_names_out()==app)[0][0]] for app in frequent_apps],width=900,height=600)
fig.update_layout(
    xaxis_title="Apps",
    yaxis_title="Number of sessions with apps")
fig.update_xaxes(type='category',tickangle = 45)

We can now calculate a similar "distribution" for each users.